<a href="https://colab.research.google.com/github/mayank2498/3dcnn_rcnn/blob/master/Minor_3D_CNN_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google_drive_downloader import GoogleDriveDownloader as gdd

activities_drive_id = '1mEnLUrnElfbqmfmQWPaBK8V-JX8zZ_sV'

gdd.download_file_from_google_drive(file_id=activities_drive_id,
                                    dest_path='./activities_public.zip',
                                    unzip=True)

Unzipping...Done.


In [0]:
!ls

In [0]:
!cp "/content/drive/My Drive/Colab Notebooks/dataset_17_20_True.npz" "/content/"

cp: cannot stat '/content/drive/My Drive/Colab Notebooks/dataset_17_20_True.npz': No such file or directory


In [0]:
!cp -r "/content/outputData" "/content/drive/My Drive/Colab Notebooks/outputData"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir outputData

In [0]:
import numpy as np
import cv2


class Videoto3D:

    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

    def video3d(self, filename, color=False, skip=True):
        
        cap = cv2.VideoCapture(filename)
        
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        
        if skip:
            frames = [x * nframe / self.depth for x in range(self.depth)]
        else:
            frames = [x for x in range(self.depth)]
        
        framearray = []

        for i in range(self.depth):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
            ret, frame = cap.read()
            
            frame = cv2.resize(frame, (self.height, self.width))
            # print("shape of frame : " ,frame.shape)
            # shape of frame :  (32,32,3)
            if color:
                framearray.append(frame)
            else:
                framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        cap.release()
        
        return np.array(framearray)/255

    def get_UCF_classname(self, filename):
        x =  filename[filename.find('_') + 1:filename.find('_', 2)]
        return x

In [0]:
import argparse
import os
import matplotlib
matplotlib.use('AGG')

import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import cifar10
from keras.layers import (Activation, Conv3D, Dense, Dropout, Flatten,
                          MaxPooling3D)

from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split

from tqdm import tqdm

from keras.callbacks import ModelCheckpoint

##
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

def plot_history(history, result_dir):
    plt.plot(history.history['acc'], marker='.')
    plt.plot(history.history['val_acc'], marker='.')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['acc', 'val_acc'], loc='lower right')
    plt.savefig(os.path.join(result_dir, 'model_accuracy.png'))
    plt.close()

    plt.plot(history.history['loss'], marker='.')
    plt.plot(history.history['val_loss'], marker='.')
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend(['loss', 'val_loss'], loc='upper right')
    plt.savefig(os.path.join(result_dir, 'model_loss.png'))
    plt.close()


def save_history(history, result_dir):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(os.path.join(result_dir, 'result.txt'), 'w') as fp:
        fp.write('epoch\tloss\tacc\tval_loss\tval_acc\n')
        for i in range(nb_epoch):
            fp.write('{}\t{}\t{}\t{}\t{}\n'.format(
                i, loss[i], acc[i], val_loss[i], val_acc[i]))
    


def loaddata(video_dir, vid3d, nclass, result_dir, color=False, skip=True):
    print("Number of classes = ",nclass)
    files = os.listdir(video_dir)    
    X = []
    labels = []
    labellist = []

    pbar = tqdm(total=len(files))

    for filename in files:
    
        pbar.update(1)
        
        if filename == '.DS_Store':
            continue
        
        name = os.path.join(video_dir, filename)

        for v_files in os.listdir(name):

            v_file_path = os.path.join(name, v_files)

            label = vid3d.get_UCF_classname(filename)
            
            if label not in labellist:
                
                if len(labellist) >= nclass:
                    continue

                labellist.append(label)
            
            labels.append(label)
            
            X.append(vid3d.video3d(v_file_path, color=color, skip=skip))

    pbar.close()

    with open(os.path.join(result_dir, 'classes.txt'), 'w') as fp:
        for i in range(len(labellist)):
            fp.write('{}\n'.format(labellist[i]))

    for num, label in enumerate(labellist):
        for i in range(len(labels)):
            if label == labels[i]:
                labels[i] = num
    if color:
        return np.array(X).transpose((0, 2, 3, 4, 1)), labels
    else:
        return np.array(X).transpose((0, 2, 3, 1)), labels


def main():
  
#     parser = argparse.ArgumentParser(
#         description='simple 3D convolution for action recognition')
#     parser.add_argument('--batch', type=int, default=128)
#     parser.add_argument('--epoch', type=int, default=100)
#     parser.add_argument('--videos', type=str, default='activities_public/',
#                         help='directory where videos are stored')
#     parser.add_argument('--nclass', type=int, default=17)
#     parser.add_argument('--output', type=str, default="outputData")
#     parser.add_argument('--color', type=bool, default=True)
#     parser.add_argument('--skip', type=bool, default=True)
#     parser.add_argument('--depth', type=int, default=10)
#     args = parser.parse_args()

    batch = 128
    epoch = 100
    videos = 'activities_public'
    nclass = 17
    output = "outputData"
    color = True
    skip = True
    depth = 20
    
    
    img_rows, img_cols, frames = 32, 32, depth
    channel = 3 if color else 1
    print("Number of worker actions available ", nclass)
    fname_npz = 'dataset_{}_{}_{}.npz'.format(
        nclass, depth, skip)
    #fname_npz = "/content/drive/My Drive/Colab Notebooks/dataset_17_20_True.npz"
    vid3d = Videoto3D(img_rows, img_cols, frames)

    if os.path.exists(fname_npz):
        loadeddata = np.load(fname_npz)
        X, Y = loadeddata["X"], loadeddata["Y"]
    else:
        x, y = loaddata(videos, vid3d, nclass,
                        output, color, skip)
        X = x.reshape((x.shape[0], img_rows, img_cols, frames, channel))
        Y = np_utils.to_categorical(y, nclass)

        X = X.astype('float32')
        np.savez(fname_npz, X=X, Y=Y)
        print('Saved dataset to dataset.npz.')
    print('X_shape:{}\nY_shape:{}'.format(X.shape, Y.shape))

    # Define model
    model = Sequential()
   
    model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(
        X.shape[1:]), border_mode='same'))

    # model.add(Activation('relu'))
    # model.add(Conv3D(32, kernel_size=(3, 3, 3), border_mode='same'))
    # model.add(Activation('softmax'))
    # model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
    # model.add(Dropout(0.25))



    model.add(Activation('relu'))
    model.add(Conv3D(32, kernel_size=(3, 3, 3), border_mode='same'))
    model.add(Activation('softmax'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
    model.add(Dropout(0.25))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), border_mode='same'))
    model.add(Activation('softmax'))
    model.add(MaxPooling3D(pool_size=(3, 3, 3), border_mode='same'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(nclass, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(), metrics=['accuracy'])
    model.summary()

    # plot_model(model, show_shapes=True,
    #            to_file=os.path.join(args.output, 'model.png'))

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=43)

    ####################

    # 1
    filepath="d_3dcnnmodel-{epoch:02d}-{val_acc:.2f}.hd5"
    
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    
    callbacks_list = [checkpoint]

    # 2 

    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.3
    set_session(tf.Session(config=config))

    ###############


    # Train model
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=batch,
                        epochs=epoch, verbose=1, shuffle=True, callbacks=callbacks_list)
    
    model.evaluate(X_test, Y_test, verbose=0)
    
    model_json = model.to_json()
    
    if not os.path.isdir(output):
        os.makedirs(output)
    with open(os.path.join(output, 'ucf101_3dcnnmodel.json'), 'w') as json_file:
        json_file.write(model_json)
    
    model.save_weights(os.path.join(output, 'ucf101_3dcnnmodel-gpu.hd5'))

    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    
    print('Test loss:', loss)
    print('Test accuracy:', acc)
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    plot_history(history, output)
    save_history(history, output)


In [8]:
main()



  0%|          | 0/17 [00:00<?, ?it/s]

Number of worker actions available  17
Number of classes =  17



100%|██████████| 17/17 [28:33<00:00, 94.20s/it] 


Saved dataset to dataset.npz.
X_shape:(8714, 32, 32, 20, 3)
Y_shape:(8714, 17)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:168: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, kernel_size=(3, 3, 3), input_shape=(32, 32, 2..., padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:179: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, kernel_size=(3, 3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:181: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(3, 3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:183: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, kernel_size=(3, 3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:185: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, kernel_size=(3, 3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:187: UserWarning: Upd



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 32, 32, 20, 32)    2624      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 20, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 32, 32, 20, 32)    27680     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 20, 32)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 11, 11, 7, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 11, 7, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 11, 11, 7, 64)  

In [0]:
!cp -r "/content/outputData" "/content/drive/My Drive/Minor/3dcnn_model_data"

In [0]:
/content/dataset_17_20_True.npz

